In [1]:
import face_recognition
import cv2
from os import listdir
from face_recognition.cli import image_files_in_folder
import os
from itertools import compress

In [2]:
# Open the input movie file
input_movie = cv2.VideoCapture("Top_Chef_S15E04.mkv")
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
length

61760

In [3]:
# Encode all faces of contestants and their names in proper format
known_faces = []
known_names = []
appearances = {}
for class_dir in listdir("Known Faces"):
    temp_path = "Known Faces/" + class_dir
    
    # Encode the faces
    temp_image = face_recognition.load_image_file(temp_path)
    known_faces.append(face_recognition.face_encodings(temp_image)[0])
    
    # Proper case names
    temp_name, ext = os.path.splitext(class_dir)
    temp_name = temp_name.replace("-"," ").title()
    known_names.append(temp_name)
    
    appearances[temp_name] = 0

In [4]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
frame_number = 0

In [5]:
while True:
    # Grab a single frame of video
    ret, frame = input_movie.read()
    frame_number += 1

    # Quit when the input video file ends
    if not ret:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)
        
        ### COUNT THE FACES ###
        
        # Get the name of the face
        name = None
        if any(match):
            name = list(compress(known_names,match))[0]
            appearances[name] += 1

        face_names.append(name)
        
    # Track progress
    if (frame_number%1000 == 0):
        print("Writing frame",frame_number,"/",length,"= {:.2%}".format(frame_number/length,"%"))

print("Done!")

Writing frame 1000 / 61760 = 1.62%
Writing frame 2000 / 61760 = 3.24%
Writing frame 3000 / 61760 = 4.86%
Writing frame 4000 / 61760 = 6.48%
Writing frame 5000 / 61760 = 8.10%
Writing frame 6000 / 61760 = 9.72%
Writing frame 7000 / 61760 = 11.33%
Writing frame 8000 / 61760 = 12.95%
Writing frame 9000 / 61760 = 14.57%
Writing frame 10000 / 61760 = 16.19%
Writing frame 11000 / 61760 = 17.81%
Writing frame 12000 / 61760 = 19.43%
Writing frame 13000 / 61760 = 21.05%
Writing frame 14000 / 61760 = 22.67%
Writing frame 15000 / 61760 = 24.29%
Writing frame 16000 / 61760 = 25.91%
Writing frame 17000 / 61760 = 27.53%
Writing frame 18000 / 61760 = 29.15%
Writing frame 19000 / 61760 = 30.76%
Writing frame 20000 / 61760 = 32.38%
Writing frame 21000 / 61760 = 34.00%
Writing frame 22000 / 61760 = 35.62%
Writing frame 23000 / 61760 = 37.24%
Writing frame 24000 / 61760 = 38.86%
Writing frame 25000 / 61760 = 40.48%
Writing frame 26000 / 61760 = 42.10%
Writing frame 27000 / 61760 = 43.72%
Writing frame 28

In [6]:
# All done!
input_movie.release()
cv2.destroyAllWindows()

In [7]:
appearances

{'Adrienne Cheatham': 934,
 'Brother Luck': 2077,
 'Bruce Kalman': 2125,
 'Carrie Baird': 1242,
 'Chris Scott': 2165,
 'Claudette Wilkins': 20,
 'Fatima Ali': 1466,
 'Joe Sasto': 2193,
 'Joseph Flamm': 1557,
 'Kwame Onwuachi': 0,
 'Laura Cole': 6,
 'Lee Anne Wong': 1,
 'Melissa Perfit': 0,
 'Rogelio Garcia': 80,
 'Tanya Holland': 1557,
 'Tu David Phu': 1362,
 'Tyler Anderson': 1493}

In [8]:
import csv

In [9]:
with open('S15E04.csv', 'w') as f:
    fieldnames = ['name', 'frames']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    data = [dict(zip(fieldnames, [k, v])) for k, v in appearances.items()]
    writer.writerows(data)